In [2]:
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.3M  100 61.3M    0     0  14.6M      0  0:00:04  0:00:04 --:--:-- 14.7M.6M


In [3]:
import pyspark


In [4]:
pyspark.__version__

'3.5.5'

In [6]:
from pyspark.sql import SparkSession


In [7]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -Djava.security.manager=allow
Picked up JAVA_TOOL_OPTIONS: -Djava.security.manager=allow
25/03/07 22:46:38 WARN Utils: Your hostname, Yes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.18.79 instead (on interface en0)
25/03/07 22:46:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 22:46:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 22:46:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
spark.version

'3.5.5'

25/03/07 22:46:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')


In [12]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [13]:
from pyspark.sql import types

In [14]:
schema = types.StructType([
    types.StructField('VendorID', types.IntegerType(), True), 
    types.StructField('tpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('tpep_dropoff_datetime', types.TimestampType(), True), 
    types.StructField('passenger_count', types.LongType(), True), 
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('RatecodeID', types.LongType(), True), 
    types.StructField('store_and_fwd_flag', types.StringType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('payment_type', types.LongType(), True), 
    types.StructField('fare_amount', types.DoubleType(), True), 
    types.StructField('extra', types.DoubleType(), True), 
    types.StructField('mta_tax', types.DoubleType(), True), 
    types.StructField('tip_amount', types.DoubleType(), True), 
    types.StructField('tolls_amount', types.DoubleType(), True), 
    types.StructField('improvement_surcharge', types.DoubleType(), True), 
    types.StructField('total_amount', types.DoubleType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True), 
    types.StructField('Airport_fee', types.DoubleType(), True)
    ])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema)\
    .parquet('yellow_tripdata_2024-10.parquet')


In [16]:
df.head(10)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 8, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 8, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 8, 12, 20), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 8, 25, 25), passenger_count=1, trip_distance=2.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=236, payment_type=1, fare_amount=14.2, extra=3.5, mta_tax=0.5, tip_amount=3.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=23.0, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 8, 4, 46), tpep_dropoff_datetime=datetime.datetime(2

In [17]:
df = df.repartition(4)

In [18]:
df.write.parquet('yellow/2024/10/')

In [19]:
df = spark.read.parquet('yellow/2024/10/')

In [22]:
from pyspark.sql import functions as F

In [20]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [30]:
df \
  .withColumn('tpep_pickup_datetime', F.to_date(df.tpep_pickup_datetime))\
  .select('tpep_pickup_datetime')\
  .show()

+--------------------+
|tpep_pickup_datetime|
+--------------------+
|          2024-10-05|
|          2024-10-04|
|          2024-10-04|
|          2024-10-02|
|          2024-10-11|
|          2024-10-03|
|          2024-10-09|
|          2024-10-10|
|          2024-10-10|
|          2024-10-05|
|          2024-10-09|
|          2024-10-11|
|          2024-10-03|
|          2024-10-04|
|          2024-10-08|
|          2024-10-06|
|          2024-10-01|
|          2024-10-09|
|          2024-10-07|
|          2024-10-07|
+--------------------+
only showing top 20 rows



In [24]:
df_filtered = df.filter(F.to_date(df.tpep_pickup_datetime) == "2024-10-15")  # Adjust year if needed


In [25]:
df_filtered.count()

116396

In [37]:
df_with_duration = df.withColumn(
    "trip_duration_hours",
    (F.unix_timestamp(F.col("tpep_dropoff_datetime")) - F.unix_timestamp(F.col("tpep_pickup_datetime"))) / 3600
)

In [38]:
max_duration = df_with_duration.select(F.max(F.col("trip_duration_hours"))).collect()[0][0]


In [39]:
max_duration

162.61777777777777

In [40]:
df_zones = spark.read.csv("taxi_zone_lookup_2.csv", header=True, inferSchema=True)


In [41]:
df_zones.head(5)

[Row(LocationID=1, Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID=2, Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID=3, Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID=4, Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID=5, Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone')]

In [42]:
df_zones.createOrReplaceTempView("taxi_zone_lookup")


In [43]:
spark.sql("SELECT * FROM taxi_zone_lookup LIMIT 5").show()


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [46]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [47]:
df.createOrReplaceTempView("yellow_taxi")

In [48]:
query = """
SELECT tz.Zone, COUNT(yt.PULocationID) AS pickup_count
FROM yellow_taxi yt
JOIN taxi_zone_lookup tz ON yt.PULocationID = tz.LocationID
GROUP BY tz.Zone
ORDER BY pickup_count ASC
LIMIT 1
"""

# Execute SQL query
least_frequent_pickup = spark.sql(query)

In [49]:
least_frequent_pickup.show()


+--------------------+------------+
|                Zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
+--------------------+------------+

